In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import datetime

#PE file

In [2]:
df1 = pd.read_excel("Excel Files/DRF-80 - Brandon Myers 2024-25 PublicEnrollment 01292025.xlsx")

In [3]:
#df1.head()

In [4]:
master_corp = pd.read_excel("Excel Files/corp_no_from_sig_dispro_calc.xlsx")

#### LEA from Sig dispor calculator





In [5]:
all_cops = master_corp['Reporting Corp'].unique()

In [6]:
#df1.PRIMARY_EXCEPTIONALITY.value_counts()

In [7]:
pd.set_option('display.max_columns', None)

In [8]:
#df1.head()

In [9]:
#df1.info()

In [10]:
df1.columns = df1.columns.str.lower()

In [11]:
df1['stn'].duplicated(keep = False).sum()

np.int64(0)

In [12]:
# duplicated_stn = df1.duplicated(keep = False)
# df2 = pd.concat([df1[duplicated_stn], df1[~duplicated_stn]])

### Saving Duplicate stn in excel

In [13]:
# 'stn' is the column containing student identification numbers
#duplicated_stn_rows = df1[df1.duplicated(subset=['stn'], keep=False)]

# Save to Excel
#duplicated_stn_rows.to_excel('duplicated_stn_with_all_fields.xlsx', index=False)

In [14]:
df1.shape

(1040190, 11)

In [15]:
df1.nunique()

school_year_id                       1
stn                            1040190
idoe_corporation_id                427
corporation_name                   427
idoe_school_id                    1891
school_name                       1808
birth_date                       10901
primary_exceptionality_code         17
primary_exceptionality              15
socio_econ_status                    3
ethnicity                            7
dtype: int64

In [16]:
#df1.describe()

### Calculate the birthdate

In [17]:
from datetime import date
df1['age'] = (date.today().year - pd.to_datetime(df1['birth_date']).dt.year)

### check the age

In [18]:
#df1.age.value_counts()

###Ethnicity algin with fedral requirment.

In [19]:
df1 = df1.replace({'White':'WH7', 'Black':'BL7', 'Hispanic': 'HI7', 'Multiracial': 'MU7', 'Asian' : 'AS7', 'American Indian' : 'AM7', 'Native Hawaiian or Other Pacific Islander' : 'PI7'})

In [20]:
#df1.ethnicity.value_counts()

In [21]:
#Flag SPED eligibility from primary exceptionality column
df1['idea-eligibility'] = np.where(df1['primary_exceptionality'].isnull(), "NonSPED", "SPED")

In [22]:
df1['idea-eligibility'].value_counts()

idea-eligibility
NonSPED    874282
SPED       165908
Name: count, dtype: int64

In [23]:
df1.shape

(1040190, 13)

In [24]:
print(874282+165908)

1040190


##School Age IDEA enrollment(5 -21)

In [25]:
sch_age = df1[
    (df1['age'] >= 5) &
    (df1['age'] <= 21)
    ]

In [26]:
#sch_age.age.value_counts()

In [27]:
sch_age.dropna(subset = ['ethnicity'], inplace = True)

C:\Users\NSinha\AppData\Local\Temp\ipykernel_27696\3849522470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sch_age.dropna(subset = ['ethnicity'], inplace = True)


In [28]:
sch_age['ethnicity'] = sch_age['ethnicity'].astype(str)

C:\Users\NSinha\AppData\Local\Temp\ipykernel_27696\3074318165.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sch_age['ethnicity'] = sch_age['ethnicity'].astype(str)


In [29]:
sch_age.columns

Index(['school_year_id', 'stn', 'idoe_corporation_id', 'corporation_name',
       'idoe_school_id', 'school_name', 'birth_date',
       'primary_exceptionality_code', 'primary_exceptionality',
       'socio_econ_status', 'ethnicity', 'age', 'idea-eligibility'],
      dtype='object')

In [30]:
sch_age = sch_age.astype({'idoe_corporation_id': 'str', 'stn': 'str'}) # converted int to string datatype

##Total enrollment form PE

In [31]:
#sch_age.info()

In [32]:
pe_total_enrolled = sch_age['idoe_corporation_id'].value_counts().reindex(all_cops, fill_value=0).reset_index()
pe_total_enrolled.columns = ['idoe_corporation_id', 'pe_total_enrolled']
#pe_total_enrolled

Enrollment By Race/Ethnicity from PE

In [33]:
pe_total_enrolled_race = sch_age.groupby('idoe_corporation_id')['ethnicity'].value_counts().unstack(fill_value=0).reindex(all_cops, fill_value=0)
pe_total_enrolled_race = pe_total_enrolled_race.reset_index()
#pe_total_enrolled_race

Merged total enrollment and race enrollment regardles of SPED

In [34]:
# Merge the total enrollment with the race-specific enrollment
pe_merged = pd.merge(pe_total_enrolled, pe_total_enrolled_race, on='idoe_corporation_id', how='left')



In [35]:
#pe_merged

#indicator 9 all other student enrolled which means total enrolled minus target race enrolled from PE file
need to count each race, then substract from total

# AM7 data

In [36]:
am7_enrl = sch_age[sch_age['ethnicity'] == 'AM7']

am7_enrl = am7_enrl.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
am7_enrl.columns = ['idoe_corporation_id', 'pe_am7_enrl']



In [37]:
# Merge the two DataFrames on 'idoe_corporation_id'
merged_enrl_am7 = pd.merge(pe_total_enrolled, am7_enrl, on='idoe_corporation_id', how='left')

# Fill NaN values in 'am7_enrolled' with 0 (for corporations with no AM7 students)
#merged_enrl_am7['pe_am7_enrl'].fillna(0, inplace=True)

# Calculate "All Other" enrollment by subtracting 'am7_enrolled' from 'pe_total_enrolled'
merged_enrl_am7['other_frm_pe_am7'] = merged_enrl_am7['pe_total_enrolled'] - merged_enrl_am7['pe_am7_enrl']

In [38]:
merged_enrl_am7 = merged_enrl_am7.drop(columns = ['pe_am7_enrl', 'pe_total_enrolled'])
merged_enrl_am7['other_frm_pe_am7'] = merged_enrl_am7['other_frm_pe_am7'].astype(int)
#merged_enrl_am7.head()

## merged AM7 data after substracting from total enrollment

In [39]:
pe_merged_am = pd.merge(pe_merged, merged_enrl_am7, on='idoe_corporation_id', how='left')
#pe_merged_am.head()

# AS7 data

In [40]:
as7_enrl = sch_age[sch_age['ethnicity'] == 'AS7']

as7_enrl = as7_enrl.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
as7_enrl.columns = ['idoe_corporation_id', 'pe_as7_enrl']

#merged
merged_as7 = pd.merge(pe_total_enrolled, as7_enrl, on='idoe_corporation_id', how='left')

merged_as7['other_frm_pe_as7'] = merged_as7['pe_total_enrolled'] - merged_as7['pe_as7_enrl']

#drop columns
merged_as7 = merged_as7.drop(columns = ['pe_as7_enrl', 'pe_total_enrolled'])
merged_as7['other_frm_pe_as7'] = merged_as7['other_frm_pe_as7'].astype(int)
pe_merged_as = pd.merge(pe_merged_am, merged_as7, on='idoe_corporation_id', how='left')
#pe_merged_as.head()





#BL7 from other

In [41]:
bl7_enrl = sch_age[sch_age['ethnicity'] == 'BL7']

bl7_enrl = bl7_enrl.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
bl7_enrl.columns = ['idoe_corporation_id', 'pe_bl7_enrl']

#merged
merged_bl7 = pd.merge(pe_total_enrolled, bl7_enrl, on='idoe_corporation_id', how='left')

merged_bl7['other_frm_pe_bl7'] = merged_bl7['pe_total_enrolled'] - merged_bl7['pe_bl7_enrl']

#drop columns
merged_bl7 = merged_bl7.drop(columns = ['pe_bl7_enrl', 'pe_total_enrolled'])
merged_bl7['other_frm_pe_bl7'] = merged_bl7['other_frm_pe_bl7'].astype(int)
pe_merged_bl = pd.merge(pe_merged_as, merged_bl7, on='idoe_corporation_id', how='left')
#pe_merged_bl.head()

#HI7 from others

In [42]:
HI7_enrl = sch_age[sch_age['ethnicity'] == 'HI7']

HI7_enrl = HI7_enrl.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
HI7_enrl.columns = ['idoe_corporation_id', 'pe_HI7_enrl']

#merged
merged_HI7 = pd.merge(pe_total_enrolled, HI7_enrl, on='idoe_corporation_id', how='left')

merged_HI7['other_frm_pe_HI7'] = merged_HI7['pe_total_enrolled'] - merged_HI7['pe_HI7_enrl']

#drop columns
merged_HI7 = merged_HI7.drop(columns = ['pe_HI7_enrl', 'pe_total_enrolled'])
merged_HI7['other_frm_pe_HI7'] = merged_HI7['other_frm_pe_HI7'].astype(int)
pe_merged_HI = pd.merge(pe_merged_bl, merged_HI7, on='idoe_corporation_id', how='left')
#pe_merged_HI.head()

#other from MU7

In [43]:
MU7_enrl = sch_age[sch_age['ethnicity'] == 'MU7']

MU7_enrl = MU7_enrl.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
MU7_enrl.columns = ['idoe_corporation_id', 'pe_MU7_enrl']

#merged
merged_MU7 = pd.merge(pe_total_enrolled, MU7_enrl, on='idoe_corporation_id', how='left')

merged_MU7['other_frm_pe_MU7'] = merged_MU7['pe_total_enrolled'] - merged_MU7['pe_MU7_enrl']

#drop columns
merged_MU7 = merged_MU7.drop(columns = ['pe_MU7_enrl', 'pe_total_enrolled'])
merged_MU7['other_frm_pe_MU7'] = merged_MU7['other_frm_pe_MU7'].astype(int)
pe_merged_MU = pd.merge(pe_merged_HI, merged_MU7, on='idoe_corporation_id', how='left')
#pe_merged_MU.head()

#Other from PI7

In [44]:
PI7_enrl = sch_age[sch_age['ethnicity'] == 'PI7']

PI7_enrl = PI7_enrl.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
PI7_enrl.columns = ['idoe_corporation_id', 'pe_PI7_enrl']

#merged
merged_PI7 = pd.merge(pe_total_enrolled, PI7_enrl, on='idoe_corporation_id', how='left')

merged_PI7['other_frm_pe_PI7'] = merged_PI7['pe_total_enrolled'] - merged_PI7['pe_PI7_enrl']

#drop columns
merged_PI7 = merged_PI7.drop(columns = ['pe_PI7_enrl', 'pe_total_enrolled'])
merged_PI7['other_frm_pe_PI7'] = merged_PI7['other_frm_pe_PI7'].astype(int)
pe_merged_PI = pd.merge(pe_merged_MU, merged_PI7, on='idoe_corporation_id', how='left')
#pe_merged_PI.head()

#Other from WH7

In [45]:
WH7_enrl = sch_age[sch_age['ethnicity'] == 'WH7']

WH7_enrl = WH7_enrl.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
WH7_enrl.columns = ['idoe_corporation_id', 'pe_WH7_enrl']

#merged
merged_WH7 = pd.merge(pe_total_enrolled, WH7_enrl, on='idoe_corporation_id', how='left')

merged_WH7['other_frm_pe_WH7'] = merged_WH7['pe_total_enrolled'] - merged_WH7['pe_WH7_enrl']

#drop columns
merged_WH7 = merged_WH7.drop(columns = ['pe_WH7_enrl', 'pe_total_enrolled'])
merged_WH7['other_frm_pe_WH7'] = merged_WH7['other_frm_pe_WH7'].astype(int)
pe_merged_WH = pd.merge(pe_merged_PI, merged_WH7, on='idoe_corporation_id', how='left')

#pe_merged_WH.head()

SE file data

In [46]:
se = pd.read_excel('Excel Files/DRF-64 - Brandon Myers Special Education 2025 December Count 01232025.xlsx')

In [47]:
se.shape

(200529, 23)

In [48]:
se.columns = se.columns.str.lower()

In [49]:
duplicated_stn = se.duplicated(keep = False)
se1  = pd.concat([se[duplicated_stn],se[~duplicated_stn]]) # put duplicated on top


In [50]:
#drop duplicate
se1.drop_duplicates(inplace = True)

In [51]:
se1.shape

(200515, 23)

In [52]:
se1.replace('KG - Morning', 'KG', inplace = True)

In [53]:
# columns convert float to integer
cols = ['reporting_corporation', 'legal_corporation', 'primaryexceptionality', 'secondaryexceptionality', 'location', 'adm_type_id']
se1[cols] = se1[cols].astype('Int64')

In [54]:
se1['grade'] = se1['grade'].astype(str)
se1['grade'] = se1['grade'].str.strip()

In [55]:
pd.set_option("display.max_columns", None)

####Excluded from the above list

*   Choice Voucher
*   Identified Unserved NonPublic/Homeschool Only
*   Private Preschool
*   Correctional Facility
*   Parentally placed in Private School

In [56]:
values_to_remove = ['Choice Voucher',
                    'Identified Unserved NonPublic/Homeschool Only',
                    'Private Preschool',
                    'Correctional Facility',
                    'Parentally placed in Private School'
]

se1= se1[~se1['programname'].isin(values_to_remove)]

In [57]:
#se1.programname.value_counts()

In [58]:
se1.replace({'White':'WH7', 'Black':'BL7', 'Hispanic': 'HI7', 'Multiracial': 'MU7', 'Asian' : 'AS7', 'American Indian' : 'AM7', 'Native Hawaiian or Other Pacific Islander' : 'PI7'}, inplace = True)

In [59]:
#se1.ethnicity.value_counts()

####Exclude location: 2, 3, 4, 10

In [60]:
se1 = se1[~se1['location'].isin([2, 3, 4, 10])]

In [61]:
#se1.location.value_counts()

In [62]:
se1.rename(columns={'reporting_corporation' : 'idoe_corporation_id'}, inplace = True)

##age 5- 21 or school age, and grade KG - 12

In [63]:
se_sch_age = se1[
    (se1['grade'].isin(['KG', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'])) &
    (se1['age_on_01_dec_2024'] >= 5) &
    (se1['age_on_01_dec_2024'] <= 21)
]

In [64]:
#se_sch_age.age_on_01_dec_2024.value_counts()

In [65]:
#se_sch_age.programname.value_counts()

In [66]:
se_sch_age = se_sch_age.astype({'idoe_corporation_id': 'str'}) # converted int to str datatype

##SE total count or total Special ed count

In [67]:
se_total_count = se_sch_age['idoe_corporation_id'].value_counts().reindex(all_cops, fill_value=0).reset_index()
se_total_count.columns = ['idoe_corporation_id', 'se_total_enrl']
#se_total_count

In [68]:
se_race_count = se_sch_age.groupby('idoe_corporation_id')['ethnicity'].value_counts().unstack(fill_value=0).reindex(all_cops, fill_value=0)
se_race_count = se_race_count.reset_index()





##merged total special ed and count by race/ethnicity of SPED

In [69]:
#se_race_count.head()

In [70]:
merged_se = pd.merge(se_total_count, se_race_count, on='idoe_corporation_id', how='left')
#merged_se.head()

#Count other SPED enrollment (Total count minus target race) -- From SE File

#AM7 data from SE

don't need to write that code each time for 7 races
copy that code and replce it
copy that code paste in the note pad and click on ctrl H replace AM to AS

In [71]:
se_am7 = se_sch_age[se_sch_age['ethnicity'] == 'AM7']
se_am7 = se_am7.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
se_am7.columns = ['idoe_corporation_id', 'se_am7_enrl']

#merged
merged_se_am7 = pd.merge(se_total_count, se_am7, on='idoe_corporation_id', how='left')
merged_se_am7['other_frm_se_am7'] = merged_se_am7['se_total_enrl'] - merged_se_am7['se_am7_enrl']

#drop columns
merged_se_am7 = merged_se_am7.drop(columns = ['se_am7_enrl', 'se_total_enrl'])
merged_se_am7['other_frm_se_am7'] = merged_se_am7['other_frm_se_am7'].astype(int)
se_merged_am = pd.merge(merged_se, merged_se_am7, on='idoe_corporation_id', how='left')
#se_merged_am.head()

#AS7 data from SE

In [72]:
se_as7 = se_sch_age[se_sch_age['ethnicity'] == 'AS7']
se_as7 = se_as7.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
se_as7.columns = ['idoe_corporation_id', 'se_as7_enrl']

#merged
merged_se_as7 = pd.merge(se_total_count, se_as7, on='idoe_corporation_id', how='left')
merged_se_as7['other_frm_se_as7'] = merged_se_as7['se_total_enrl'] - merged_se_as7['se_as7_enrl']

#drop columns
merged_se_as7 = merged_se_as7.drop(columns = ['se_as7_enrl', 'se_total_enrl'])
merged_se_as7['other_frm_se_as7'] = merged_se_as7['other_frm_se_as7'].astype(int)
se_merged_as = pd.merge(se_merged_am, merged_se_as7, on='idoe_corporation_id', how='left')
#se_merged_as.head()

#BL7 data from SE

In [73]:
se_bl7 = se_sch_age[se_sch_age['ethnicity'] == 'BL7']
se_bl7 = se_bl7.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
se_bl7.columns = ['idoe_corporation_id', 'se_bl7_enrl']

#merged
merged_se_bl7 = pd.merge(se_total_count, se_bl7, on='idoe_corporation_id', how='left')
merged_se_bl7['other_frm_se_bl7'] = merged_se_bl7['se_total_enrl'] - merged_se_bl7['se_bl7_enrl']

#drop columns
merged_se_bl7 = merged_se_bl7.drop(columns = ['se_bl7_enrl', 'se_total_enrl'])
merged_se_bl7['other_frm_se_bl7'] = merged_se_bl7['other_frm_se_bl7'].astype(int)
se_merged_bl = pd.merge(se_merged_as, merged_se_bl7, on='idoe_corporation_id', how='left')
#se_merged_bl.head()

#HI7 from SE

In [74]:
se_hi7 = se_sch_age[se_sch_age['ethnicity'] == 'HI7']
se_hi7 = se_hi7.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
se_hi7.columns = ['idoe_corporation_id', 'se_hi7_enrl']

#merged
merged_se_hi7 = pd.merge(se_total_count, se_hi7, on='idoe_corporation_id', how='left')
merged_se_hi7['other_frm_se_hi7'] = merged_se_hi7['se_total_enrl'] - merged_se_hi7['se_hi7_enrl']

#drop columns
merged_se_hi7 = merged_se_hi7.drop(columns = ['se_hi7_enrl', 'se_total_enrl'])
merged_se_hi7['other_frm_se_hi7'] = merged_se_hi7['other_frm_se_hi7'].astype(int)
se_merged_hi = pd.merge(se_merged_bl, merged_se_hi7, on='idoe_corporation_id', how='left')
#se_merged_hi.head()

#MU data from SE

In [75]:
se_mu7 = se_sch_age[se_sch_age['ethnicity'] == 'MU7']
se_mu7 = se_mu7.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
se_mu7.columns = ['idoe_corporation_id', 'se_mu7_enrl']

#merged
merged_se_mu7 = pd.merge(se_total_count, se_mu7, on='idoe_corporation_id', how='left')
merged_se_mu7['other_frm_se_mu7'] = merged_se_mu7['se_total_enrl'] - merged_se_mu7['se_mu7_enrl']

#drop columns
merged_se_mu7 = merged_se_mu7.drop(columns = ['se_mu7_enrl', 'se_total_enrl'])
merged_se_mu7['other_frm_se_mu7'] = merged_se_mu7['other_frm_se_mu7'].astype(int)
se_merged_mu = pd.merge(se_merged_hi, merged_se_mu7, on='idoe_corporation_id', how='left')
#se_merged_mu.head()


#PI data from SE

In [76]:
se_pi7 = se_sch_age[se_sch_age['ethnicity'] == 'PI7']
se_pi7 = se_pi7.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
se_pi7.columns = ['idoe_corporation_id', 'se_pi7_enrl']

#merged
merged_se_pi7 = pd.merge(se_total_count, se_pi7, on='idoe_corporation_id', how='left')
merged_se_pi7['other_frm_se_pi7'] = merged_se_pi7['se_total_enrl'] - merged_se_pi7['se_pi7_enrl']

#drop columns
merged_se_pi7 = merged_se_pi7.drop(columns = ['se_pi7_enrl', 'se_total_enrl'])
merged_se_pi7['other_frm_se_pi7'] = merged_se_pi7['other_frm_se_pi7'].astype(int)
se_merged_pi = pd.merge(se_merged_mu, merged_se_pi7, on='idoe_corporation_id', how='left')
#se_merged_pi.head()

#WH data from SE

In [77]:
se_wh7 = se_sch_age[se_sch_age['ethnicity'] == 'WH7']
se_wh7 = se_wh7.idoe_corporation_id.value_counts().reindex(all_cops, fill_value=0).reset_index()
se_wh7.columns = ['idoe_corporation_id', 'se_wh7_enrl']

#merged
merged_se_wh7 = pd.merge(se_total_count, se_wh7, on='idoe_corporation_id', how='left')
merged_se_wh7['other_frm_se_wh7'] = merged_se_wh7['se_total_enrl'] - merged_se_wh7['se_wh7_enrl']

#drop columns
merged_se_wh7 = merged_se_wh7.drop(columns = ['se_wh7_enrl', 'se_total_enrl'])
merged_se_wh7['other_frm_se_wh7'] = merged_se_wh7['other_frm_se_wh7'].astype(int)
se_merged_wh = pd.merge(se_merged_pi, merged_se_wh7, on='idoe_corporation_id', how='left')
#se_merged_wh.head()



#merged PE file to SE file for indicator 9 caculation

In [78]:
#se_merged_wh.head()

##Rename race/ethnicity columns before merging to PE file

In [79]:
se_merged = se_merged_wh.rename(columns = {'AM7': 'SE_AM7', 'AS7': 'SE_AS7', 'BL7': 'SE_BL7', 'HI7': 'SE_HI7', 'MU7': 'SE_MU7', 'PI7': 'SE_PI7', 'WH7': 'SE_WH7'})

In [80]:
#se_merged_wh.head()

In [81]:
all_merged = pd.merge(pe_merged_WH, se_merged, on='idoe_corporation_id', how='left')

In [82]:
#all_merged.head()

#Indicator 9 calculation

Target Risk : Black student with disability / Black student enrolled

Other Risk: All other Students with disability  divided by All other Students enrolled.

Risk ratio =  Target Risk divided by Other Risk

Black student with disability – N Size =  15

Black student enrolled Cell Size = 15

All other Students with disability  N Size = 15

All other Students enrolled. cell Size = 15




In [83]:
all_merged.shape

(511, 31)

##Risk ratio American Indian (AM7)

In [84]:
all_merged['risk_ratio_am7'] = (all_merged['SE_AM7'] * all_merged['other_frm_pe_am7']) / (all_merged['AM7'] * all_merged['other_frm_se_am7'])
all_merged['risk_ratio_am7'] = all_merged['risk_ratio_am7'].replace([np.inf, -np.inf], np.nan)

##Risk ratio Asian (AS7)

In [85]:
all_merged['risk_ratio_as7'] = (all_merged['SE_AS7'] * all_merged['other_frm_pe_as7']) / (all_merged['AS7'] * all_merged['other_frm_se_as7'])
all_merged['risk_ratio_as7'] = all_merged['risk_ratio_as7'].replace([np.inf, -np.inf], np.nan)

##Risk ratio Black(BL7)

In [86]:
all_merged['risk_ratio_bl7'] = (all_merged['SE_BL7'] * all_merged['other_frm_pe_bl7']) / (all_merged['BL7'] * all_merged['other_frm_se_bl7'])
all_merged['risk_ratio_bl7'] = all_merged['risk_ratio_bl7'].replace([np.inf, -np.inf], np.nan)

##Risk ratio Hispanic(HI7)

In [87]:
all_merged['risk_ratio_hi7'] = (all_merged['SE_HI7'] * all_merged['other_frm_pe_HI7']) / (all_merged['HI7'] * all_merged['other_frm_se_hi7'])
all_merged['risk_ratio_hi7'] = all_merged['risk_ratio_hi7'].replace([np.inf, -np.inf], np.nan)

##Risk ratio Multiracial (Mu7)

In [88]:
all_merged['risk_ratio_mu7'] = (all_merged['SE_MU7'] * all_merged['other_frm_pe_MU7']) / (all_merged['MU7'] * all_merged['other_frm_se_mu7'])
all_merged['risk_ratio_mu7'] = all_merged['risk_ratio_mu7'].replace([np.inf, -np.inf], np.nan)

##Risk ratio Native Hawaiian or Other Pacific Islander(PI7)

In [89]:
all_merged['risk_ratio_pi7'] = (all_merged['SE_PI7'] * all_merged['other_frm_pe_PI7']) / (all_merged['PI7'] * all_merged['other_frm_se_pi7'])
all_merged['risk_ratio_pi7'] = all_merged['risk_ratio_pi7'].replace([np.inf, -np.inf], np.nan)

##Risk ratio White(WH)

In [90]:
all_merged['risk_ratio_wh7'] = (all_merged['SE_WH7'] * all_merged['other_frm_pe_WH7']) / (all_merged['WH7'] * all_merged['other_frm_se_wh7'])
all_merged['risk_ratio_wh7'] = all_merged['risk_ratio_wh7'].replace([np.inf, -np.inf], np.nan)

In [91]:
#all_merged.head()

In [92]:
#all_merged.to_excel('all_merged_indicator_9_calc.xlsx', index = False)

#Data extraction for indicator 10 calculation

Target risk : Black student with ED divided by black student enrolled

Other risk : All other identified as ED divided by All other student enrolled

Target risk : Black student with ED divided by black student enrolled

Other risk : All other identified as ED divided by All other student enrolled

Six specific disabilities:
1.       Autism
2.       Emotional Disturbance(ED)
3.       Intellectual disability
4.       Other health Impairments
5.       Specific Learning Disability and
6.     Speech or language impairment

need to extract each specific disability with each target race. that data need to mege with pe_merged_WH


Run the disability code file, merge with se file to get disability code

In [93]:
disability = pd.read_excel('Excel Files/Disability_codes.xlsx')

In [94]:
#disability.head(20)

Converted full time and part time disability to Emotional disability
mild, moderate, and severe intellectual disability to intellectual disability


In [95]:
disability.replace('Emotional Disability (Full-Time)', 'Emotional Disturbance', inplace = True)
disability.replace('Emotional Disability (Part-Time)', 'Emotional Disturbance', inplace = True)

In [96]:
disability.replace('Mild Intellectual Disability', 'Intellectual Disability', inplace = True)
disability.replace('Moderate Intellectual Disability', 'Intellectual Disability', inplace = True)
disability.replace('Severe Intellectual Disability', 'Intellectual Disability', inplace = True)

In [97]:
#disability.head(20)

In [98]:
se_dis_sch_age = se_sch_age.merge(disability, right_on = 'Code', left_on = 'primaryexceptionality', how = 'left')

In [99]:
se_dis_sch_age.columns

Index(['schoolyear', 'submissionwindowid', 'idoe_corporation_id',
       'reporting_corporation_name', 'legal_corporation',
       'legal_corporation_name', 'idoe_school_id', 'school_name', 'stn',
       'gender', 'ethnicity', 'birth_date', 'age_on_01_dec_2024', 'grade',
       'programname', 'primaryexceptionality', 'secondaryexceptionality',
       'primaryfundinglevel', 'secondaryfundinglevel', 'ideaeligibility',
       'socio_econ_status_category', 'location', 'adm_type_id', 'Code',
       'Disability'],
      dtype='object')

In [100]:
#se_dis_sch_age.Disability.value_counts()

In [101]:
from itertools import product
ETHNICITY_CODES = {
    'AM7': 'American Indian',
    'HI7': 'Hispanic',
    'AS7': 'Asian',
    'MU7': 'Multiracial',
    'BL7': 'Black',
    'WH7': 'White',
    'PI7': 'Other Pacific Islander'
}

DISABILITY_TYPES = [
    'Autism Spectrum Disorder',
    'Emotional Disturbance',
    'Intellectual Disability',
    'Other Health Impairment',
    'Specific Learning Disability',
    'Speech/Language Impairment'
]



# 3. Get unique corporation IDs
corporation_ids = se_dis_sch_age['idoe_corporation_id'].unique()

# 4. risk ratio function for corporation-level calculation
def calculate_corporation_rr(corp_id, target_ethnicity, target_disability):
    #Calculate risk ratio for a specific corporation#
    # Filter data for this corporation
    corp_sch_age = sch_age[sch_age['idoe_corporation_id'] == corp_id]
    corp_disabilities = se_dis_sch_age[se_dis_sch_age['idoe_corporation_id'] == corp_id]

    # Get sch_age counts
    B = corp_sch_age[corp_sch_age['ethnicity'] == target_ethnicity].shape[0]
    D = corp_sch_age[corp_sch_age['ethnicity'] != target_ethnicity].shape[0]

    # Get disability counts
    A = corp_disabilities[(corp_disabilities['ethnicity'] == target_ethnicity) &
                         (corp_disabilities['Disability'] == target_disability)].shape[0]

    C = corp_disabilities[(corp_disabilities['ethnicity'] != target_ethnicity) &
                         (corp_disabilities['Disability'] == target_disability)].shape[0]

    # Calculate risk ratio
    try:
        return (A * D) / (B * C) if (B * C) != 0 else float('nan')
    except:
        return float('nan')

# 5. Calculate all combinations for all corporations
results = []
for corp_id in corporation_ids:
    for eth_code, disability in product(ETHNICITY_CODES.keys(), DISABILITY_TYPES):
        rr = calculate_corporation_rr(corp_id, eth_code, disability)
        results.append({
            'Corporation_ID': corp_id,
            'Ethnicity_Code': eth_code,
            'Ethnicity': ETHNICITY_CODES[eth_code],
            'Disability': disability,
            'Risk_Ratio': rr
        })

# 6. Create results DataFrame
results_df = pd.DataFrame(results)

# 7. Save comprehensive results
results_df.to_csv('indicator_10_risk_ratios.csv', index=False)

# 8. Create a summary pivot table
pivot_summary = results_df.pivot_table(
    index=['Corporation_ID', 'Ethnicity'],
    columns='Disability',
    values='Risk_Ratio'
)

# 9. Save and display summary
# pivot_summary.to_csv('indicator_10_risk_ratios_summary.csv')
# print("Top 10 Corporation Risk Ratios:")
# display(pivot_summary.head())


#Validation of A, B, C, and D for random selection
Changing Ethnicity and disabilites and verifying exact filter.

In [102]:
# se_am_autism = se_dis_sch_age[(se_dis_sch_age['ethnicity'] == 'MU7') & (se_dis_sch_age['Disability'] == 'Specific Learning Disability')]

# ch1 = se_am_autism.query("idoe_corporation_id == 370")
#ch1 # A filtered numerator A filter accuratly

Denominator validation (B)

In [103]:
# filtered_sch_age = sch_age[
#     (sch_age['idoe_corporation_id'] == 370) &
#     (sch_age['ethnicity'] == 'AM7') &
#     (sch_age['idea-eligibility'] == 'NonSPED')
# ]

In [104]:
#filtered_sch_age.ethnicity.value_counts()

In [105]:
# check leading speces
leading_spaces = sch_age['ethnicity'].str.startswith(' ').any()
#print(f"leading spcaces: {leading_spaces}")


In [106]:
#print(se_am_autism['idoe_corporation_id'].unique())

In [107]:
#se_am_autism.info()

#Verification from main data source

In [108]:
#ch2 = se_dis_sch_age.query("idoe_corporation_id == 370")
#ch2

#Denominator and Numerator data
Meets the minimum cell and n size

In [109]:
# from itertools import product
# ETHNICITY_CODES = {
#     'AM7': 'American Indian',
#     'HI7': 'Hispanic',
#     'AS7': 'Asian',
#     'MU7': 'Multiracial',
#     'BL7': 'Black',
#     'WH7': 'White',
#     'PI7': 'Other Pacific Islander'
# }

# DISABILITY_TYPES = [
#     'Autism Spectrum Disorder',
#     'Emotional Disturbance',
#     'Intellectual Disability',
#     'Other Health Impairment',
#     'Specific Learning Disability',
#     'Speech/Language Impairment'
# ]

# MIN_CELL_SIZE = 15  # Your threshold for both numerator and denominator

# # 3. Calculate corporation-level risk ratios with components
# def calculate_corporation_rr(corp_id, target_eth, target_dis):
#     """Calculate with all components and size checks"""
#     # Filter to corporation
#     corp_enroll =sch_age[sch_age['idoe_corporation_id'] == corp_id]
#     corp_dis = se_dis_sch_age[se_dis_sch_age['idoe_corporation_id'] == corp_id]

#     # Calculate components
#     A = corp_dis[(corp_dis['ethnicity'] == target_eth) &
#                 (corp_dis['Disability'] == target_dis)].shape[0]
#     B = corp_enroll[corp_enroll['ethnicity'] == target_eth].shape[0]
#     C = corp_dis[(corp_dis['ethnicity'] != target_eth) &
#                 (corp_dis['Disability'] == target_dis)].shape[0]
#     D = corp_enroll[corp_enroll['ethnicity'] != target_eth].shape[0]

#     # Check cell size criteria
#     meets_criteria = all(x >= MIN_CELL_SIZE for x in [A, B, C, D])

#     # Calculate risk ratio if criteria met
#     rr = (A * D) / (B * C) if meets_criteria and (B * C) != 0 else float('nan')

#     return {
#         'A': A, 'B': B, 'C': C, 'D': D,
#         'meets_criteria': meets_criteria,
#         'Risk_Ratio': rr
#     }

# # 4. Process all combinations
# results = []
# for corp_id in se_dis_sch_age['idoe_corporation_id'].unique():
#     for eth_code, disability in product(ETHNICITY_CODES.keys(), DISABILITY_TYPES):
#         calc = calculate_corporation_rr(corp_id, eth_code, disability)

#         results.append({
#             'Corporation_ID': corp_id,
#             'Ethnicity_Code': eth_code,
#             'Ethnicity': ETHNICITY_CODES[eth_code],
#             'Disability': disability,
#             **calc  # Unpacks all calculated values
#         })

# # 5. Create DataFrame
# results_df = pd.DataFrame(results)

# # 6. Analysis of meeting criteria
# criteria_met = results_df['meets_criteria'].sum()
# total_combinations = len(results_df)
# print(f"\nCell size criteria met in {criteria_met}/{total_combinations} cases ({criteria_met/total_combinations:.1%})")

# # 7. Save detailed results
# output_cols = [
#     'Corporation_ID', 'Ethnicity', 'Disability',
#     'A', 'B', 'C', 'D', 'meets_criteria', 'Risk_Ratio'
# ]
# results_df[output_cols].to_csv('corporation_rr_with_components.csv', index=False)

# 8. Create filtered version (only valid calculations)
valid_results = results_df[results_df['meets_criteria']].copy()
#valid_results.to_csv('valid_corporation_risk_ratios.csv', index=False)

#print("\nSample of valid results:")
#display(valid_results.head())


KeyError: 'meets_criteria'

#Fail to meet the minimum n and cell sizes

In [ ]:
# from itertools import product
# ETHNICITY_CODES = {
#     'AM7': 'American Indian',
#     'HI7': 'Hispanic',
#     'AS7': 'Asian',
#     'MU7': 'Multiracial',
#     'BL7': 'Black',
#     'WH7': 'White',
#     'PI7': 'Other Pacific Islander'
# }

# DISABILITY_TYPES = [
#     'Autism Spectrum Disorder',
#     'Emotional Disturbance',
#     'Intellectual Disability',
#     'Other Health Impairment',
#     'Specific Learning Disability',
#     'Speech/Language Impairment'
# ]



# def calculate_with_flags(corp_id, eth, dis):
#     """Calculate components and flag any that fail size criteria"""
#     # Filter to corporation
#     corp_enroll = sch_age[sch_age['idoe_corporation_id'] == corp_id]
#     corp_dis = se_dis_sch_age[se_dis_sch_age['idoe_corporation_id'] == corp_id]

#     # Calculate components
#     A = corp_dis[(corp_dis['ethnicity'] == eth) & (corp_dis['Disability'] == dis)].shape[0]
#     B = corp_enroll[corp_enroll['ethnicity'] == eth].shape[0]
#     C = corp_dis[(corp_dis['ethnicity'] != eth) & (corp_dis['Disability'] == dis)].shape[0]
#     D = corp_enroll[corp_enroll['ethnicity'] != eth].shape[0]

#     # Flag components that fail criteria
#     flags = {
#         'A_Valid': A >= MIN_CELL_SIZE,
#         'B_Valid': B >= MIN_CELL_SIZE,
#         'C_Valid': C >= MIN_CELL_SIZE,
#         'D_Valid': D >= MIN_CELL_SIZE,
#     }

#     # Overall validity
#     meets_criteria = all(flags.values())

#     # Calculate RR only if valid
#     rr = (A * D) / (B * C) if meets_criteria and (B * C) != 0 else float('nan')

#     return {
#         'A': A, 'B': B, 'C': C, 'D': D,
#         **flags,
#         'All_Components_Valid': meets_criteria,
#         'Risk_Ratio': rr,
#         'Note': "" if meets_criteria else "Excluded - Fails cell size criteria"
#     }

# # Process all combinations
# flagged_results = []
# for corp_id in se_dis_sch_age['idoe_corporation_id'].unique():
#     for eth, dis in product(ETHNICITY_CODES.keys(), DISABILITY_TYPES):
#         res = calculate_with_flags(corp_id, eth, dis)
#         flagged_results.append({
#             'Corporation_ID': corp_id,
#             'Ethnicity_Code': eth,
#             'Ethnicity': ETHNICITY_CODES[eth],
#             'Disability': dis,
#             **res
#         })

# # Create DataFrame
# flagged_df = pd.DataFrame(flagged_results)

# # Highlight problematic corporations
# problem_corporations = flagged_df[~flagged_df['All_Components_Valid']]['Corporation_ID'].unique()
# print(f"Corporations needing review ({len(problem_corporations)}): {sorted(problem_corporations)}")

# # Save with conditional formatting hints
# with pd.ExcelWriter('flagged_risk_ratios.xlsx') as writer:
#     # Full data
#     flagged_df.to_excel(writer, sheet_name='All Calculations', index=False)

#     # Valid only
#     valid_df = flagged_df[flagged_df['All_Components_Valid']]
#     valid_df.to_excel(writer, sheet_name='Valid Calculations', index=False)

#     # Problem cases
#     problem_df = flagged_df[~flagged_df['All_Components_Valid']]
#     problem_df.to_excel(writer, sheet_name='Needs Review', index=False)

#print("Output saved to 'flagged_risk_ratios.xlsx' with separate tabs")


All RR - that data used in the final submission

In [ ]:
# from itertools import product
# ETHNICITY_CODES = {
#     'AM7': 'American Indian',
#     'HI7': 'Hispanic',
#     'AS7': 'Asian',
#     'MU7': 'Multiracial',
#     'BL7': 'Black',
#     'WH7': 'White',
#     'PI7': 'Other Pacific Islander'
# }

# DISABILITY_TYPES = [
#     'Autism Spectrum Disorder',
#     'Emotional Disturbance',
#     'Intellectual Disability',
#     'Other Health Impairment',
#     'Specific Learning Disability',
#     'Speech/Language Impairment'
# ]


# # 2. Configuration
# MIN_CELL_SIZE = 15  # Your threshold

# # 3. Enhanced calculation function
# def calculate_rr_with_components(corp_id, eth, dis):
#     corp_enroll = sch_age[sch_age['idoe_corporation_id'] == corp_id]
#     corp_dis = se_dis_sch_age[se_dis_sch_age['idoe_corporation_id'] == corp_id]

#     # Calculate components
#     A = corp_dis[(corp_dis['ethnicity'] == eth) & (corp_dis['Disability'] == dis)].shape[0]
#     B = corp_enroll[corp_enroll['ethnicity'] == eth].shape[0]
#     C = corp_dis[(corp_dis['ethnicity'] != eth) & (corp_dis['Disability'] == dis)].shape[0]
#     D = corp_enroll[corp_enroll['ethnicity'] != eth].shape[0]

#     # Check criteria
#     valid_A = A >= MIN_CELL_SIZE
#     valid_B = B >= MIN_CELL_SIZE
#     valid_C = C >= MIN_CELL_SIZE
#     valid_D = D >= MIN_CELL_SIZE
#     meets_criteria = all([valid_A, valid_B, valid_C, valid_D])

#     # Calculate RR only if meets criteria
#     rr = (A * D) / (B * C) if meets_criteria and (B * C) != 0 else float('nan')

#     return {
#         'A': A, 'B': B, 'C': C, 'D': D,
#         'meets_criteria': meets_criteria,
#         'valid_A': valid_A, 'valid_B': valid_B, 'valid_C': valid_C, 'valid_D': valid_D,
#         'Risk_Ratio': rr
#     }

# # 4. Process all combinations (keeping ALL results)
# full_results = []
# for corp_id in se_dis_sch_age['idoe_corporation_id'].unique():
#     for eth, dis in product(ETHNICITY_CODES.keys(), DISABILITY_TYPES):
#         res = calculate_rr_with_components(corp_id, eth, dis)
#         full_results.append({
#             'Corporation_ID': corp_id,
#             'Ethnicity': ETHNICITY_CODES[eth],
#             'Disability': dis,
#             **res
#         })

# # 5. Create DataFrames
# all_results_df = pd.DataFrame(full_results)  # Contains ALL calculations
# valid_results_df = all_results_df[all_results_df['meets_criteria']].copy()  # Only valid

# # 6. Analysis reports
# print("Data Quality Report:")
# print(f"Total calculations: {len(all_results_df)}")
# print(f"Valid calculations (meets all criteria): {len(valid_results_df)} ({len(valid_results_df)/len(all_results_df):.1%})")
# print("\nBreakdown of failing criteria:")
# criteria_report = all_results_df[['valid_A', 'valid_B', 'valid_C', 'valid_D']].apply(lambda x: x.value_counts())
# display(criteria_report)

# # 7. round risk ratio 4 after decimal, A, B, C, D should be whole number
# valid_results_df = valid_results_df.round({
#     'A': 0, 'B': 0, 'C': 0, 'D': 0, 'Risk_Ratio': 4
# })

# # Save outputs
# all_results_df.to_csv('ALL_corporation_RR_calculations.csv', index=False)  # All attempts
# valid_results_df.to_csv('VALID_corporation_risk_ratios.csv', index=False)  # Only valid

# # 8. Optional: Create troubleshooting file
# problem_cases = all_results_df[~all_results_df['meets_criteria']]
# problem_cases.to_csv('corporations_needing_review.csv', index=False)


#Risk Ratio validation

In [ ]:
#Validation Function
# rr_validation = pd.read_csv('ALL_corporation_RR_calculations.csv')
# def valdiate_row(row):
#   try:
#     #Recalculate from component
#     recalculated_rr = (row['A'] * row['D']) / (row['B'] * row['C'])
#     #handle divsion by zero errors
#     if pd.isna(recalculated_rr) and pd.isna(row['Risk_Ratio']):
#       return True
#       #Compare with tolrence for floating point differences
#       return abs(recalculated_rr - row['Risk_Ratio']) < 0.000001
#   except:
#     return False

# #Apply validation
# rr_validation['Validation_Result'] = rr_validation.apply(valdiate_row, axis=1)


#Numerator and denominator validation

In [ ]:
sch_age.info()

In [ ]:
se_dis_sch_age.info()

In [ ]:
# import pandas as pd

# # 1. Load data with explicit dtype for IDs to avoid mixed-type issues
# enrollment = pd.read_csv('sch_age.csv', dtype={'idoe_corporation_id': str, 'stn': str})
# disabilities = pd.read_csv('se_dis_sch_age.csv', dtype={'idoe_corporation_id': str, 'STn': str})

# # 2. Validation function for Corporation X and Ethnicity Y
# def validate_counts(corp_id, eth_code):
#     print(f"\n Validation for Corporation {corp_id}, Ethnicity {eth_code} ({ETHNICITY_CODES[eth_code]})")

#     # Get ACTUAL enrollment (B) from raw data
#     actual_B = enrollment[
#         (enrollment['idoe_corporation_id'] == corp_id) &
#         (enrollment['ethnicity'] == eth_code)
#     ].shape[0]

#     # Get what the RR calculation was using
#     calc_B = flagged_df[
#         (flagged_df['Corporation_ID'] == corp_id) &
#         (flagged_df['Ethnicity_Code'] == eth_code)
#     ]['B'].values[0] if not flagged_df[
#         (flagged_df['Corporation_ID'] == corp_id) &
#         (flagged_df['Ethnicity_Code'] == eth_code)
#     ].empty else 0

#     print(f"• Enrollment from raw data: {actual_B}")
#     print(f"• Enrollment used in calc: {calc_B}")
#     print(f"• Match: {actual_B == calc_B}")

# # 3. Spot-check problematic cases
# print("🛠️ Debugging Enrollment Counts (B values):")

# # Case 1: Check a corporation where you noticed discrepancies
# validate_counts('123', 'BL7')  # Replace with actual problematic corp ID

# # Case 2: Check a case that worked correctly
# validate_counts('456', 'WH7')  # Replace with known good corp ID

# # 4. Recalculate B values with verification
# def recalculate_B_values():
#     """Recompute enrollment counts with verification checks"""
#     for corp_id in disabilities['idoe_corporation_id'].unique():
#         corp_enroll = enrollment[enrollment['idoe_corporation_id'] == corp_id]

#         for eth_code in ETHNICITY_CODES:
#             # New calculation
#             true_count = corp_enroll[corp_enroll['ethnicity'] == eth_code].shape[0]

#             # Update all relevant rows in flagged_df
#             mask = (flagged_df['Corporation_ID'] == corp_id) & (flagged_df['Ethnicity_Code'] == eth_code)
#             flagged_df.loc[mask, 'B'] = true_count

#             # Revalidate
#             if not mask.any():
#                 print(f" No entries for Corp {corp_id}, Ethnicity {eth_code}")

#     print(" B values recalculated and verified")

# # 5. Execute recalculation
# recalculate_B_values()

# # 6. Recheck previously problematic cases
# print("\n Post-fix verification:")
# validate_counts('123', 'BL7')  # Same corp as before

# # 7. Save corrected data
# flagged_df.to_csv('corrected_risk_ratios.csv', index=False)
# print("\n Saved corrected data to 'corrected_risk_ratios.csv'")

# # Check for duplicate student IDs that might affect counts
#    print(f"Duplicate STn in enrollment: {enrollment['STn'].duplicated().sum()}")

#    # Check ethnicity value distribution
#    print(enrollment['ethnicity'].value_counts())

# # Deep dive into specific corporation
#    corp = '123'  # Problematic corporation
#    print("\nEnrollment records for Corp", corp)
#    display(enrollment[enrollment['idoe_corporation_id'] == corp]['ethnicity'].value_counts())

#    print("\nDisability records for Corp", corp)
#    display(disabilities[disabilities['idoe_corporation_id'] == corp]['ethnicity'].value_counts())

# Expected output

#  Debugging Enrollment Counts (B values):

#  Validation for Corporation 123, Ethnicity BL7 (Black)
# • Enrollment from raw data: 42
# • Enrollment used in calc: 38
# • Match: False

# B values recalculated and verified

# Post-fix verification:
#  Validation for Corporation 123, Ethnicity BL7 (Black)
# • Enrollment from raw data: 42
# • Enrollment used in calc: 42
# • Match: True



In [ ]:
# df = pd.read_csv('ALL_corporation_RR_calculations.csv')

# # Validation function
# def validate_row(row):
#     try:
#         # Recalculate from components
#         recalculated_rr = (row['A'] * row['D']) / (row['B'] * row['C'])

#         # Handle division by zero
#         if pd.isna(recalculated_rr) and pd.isna(row['Risk_Ratio']):
#             return True

#         # Compare with tolerance for floating point differences
#         return abs(recalculated_rr - row['Risk_Ratio']) < 0.000001
#     except:
#         return False

# # Apply validation
# df['Validation_Result'] = df.apply(validate_row, axis=1)

# # Check discrepancies
# discrepancies = df[~df['Validation_Result']]
# print(f"Found {len(discrepancies)} discrepancies out of {len(df)} calculations")

# # Save validation report
# df.to_csv('VALIDATED_corporation_RR.csv', index=False)

# Spot check specific cases:
# # Function for manual spot-checking
# def spot_check(corp_id, ethnicity, disability):
#     subset = df[
#         (df['Corporation_ID'] == corp_id) &
#         (df['Ethnicity'] == ethnicity) &
#         (df['Disability'] == disability)
#     ]
#     if len(subset) == 0:
#         print("No matching records found")
#         return

#     row = subset.iloc[0]
#     print(f"\n Spot Check for {ethnicity} students with {disability} at Corporation {corp_id}:")
#     print(f"A (numerator): {row['A']} {ethnicity} students with {disability}")
#     print(f"B (denominator): {row['B']} total {ethnicity} students")
#     print(f"C: {row['C']} non-{ethnicity} students with {disability}")
#     print(f"D: {row['D']} total non-{ethnicity} students")
#     print(f"\nCalculation: ({row['A']} × {row['D']}) / ({row['B']} × {row['C']})")
#     print(f"Stored RR: {row['Risk_Ratio']:.4f}")
#     print(f"Recalculated: {(row['A'] * row['D']) / (row['B'] * row['C']):.4f}")

# # Example usage - check a known case
# spot_check(corp_id=123, ethnicity='Black', disability='autism')

# Statistical sanity check
# # 1. Check RR distribution
# print("\n Risk Ratio Distribution:")
# print(df['Risk_Ratio'].describe())

# # 2. Verify extreme values
# print("\n🔝 Top 10 Highest Risk Ratios:")
# print(df.nlargest(10, 'Risk_Ratio')[['Corporation_ID', 'Ethnicity', 'Disability', 'Risk_Ratio']])

# # 3. Check zeros/infinities
# print("\n Special Cases:")
# print(f"Zero RR: {len(df[df['Risk_Ratio'] == 0])} cases")
# print(f"Infinite RR: {len(df[df['Risk_Ratio'] == float('inf')])} cases")
# print(f"NaN RR: {len(df[pd.isna(df['Risk_Ratio'])])} cases")

# Visualization for outliers
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 6))
# plt.scatter(df['A'], df['Risk_Ratio'], alpha=0.5)
# plt.xlabel('Numerator (A)')
# plt.ylabel('Risk Ratio')
# plt.title('Risk Ratio vs. Numerator Size')
# plt.yscale('log')  # Helps visualize wide ranges
# plt.grid(True)
# plt.savefig('rr_validation_plot.png')
# plt.show()

# Export validation report

# with pd.ExcelWriter('validation_report.xlsx') as writer:
#     # All calculations with validation flag
#     df.to_excel(writer, sheet_name='All Calculations', index=False)

#     # Discrepancies only
#     discrepancies.to_excel(writer, sheet_name='Discrepancies', index=False)

#     # Summary stats
#     pd.DataFrame({
#         'Validation Metric': ['Total Calculations', 'Valid Calculations', 'Invalid Calculations'],
#         'Count': [len(df), len(df[df['Validation_Result']]), len(discrepancies)]
#     }).to_excel(writer, sheet_name='Summary', index=False)

# print(" Validation report saved to validation_report.xlsx")


In [110]:
from itertools import product
ETHNICITY_CODES = {
    'AM7': 'American Indian',
    'HI7': 'Hispanic',
    'AS7': 'Asian',
    'MU7': 'Multiracial',
    'BL7': 'Black',
    'WH7': 'White',
    'PI7': 'Other Pacific Islander'
}

DISABILITY_TYPES = [
    'Autism Spectrum Disorder',
    'Emotional Disturbance',
    'Intellectual Disability',
    'Other Health Impairment',
    'Specific Learning Disability',
    'Speech/Language Impairment'
]


# # 2. Configuration
MIN_CELL_SIZE = 15  # Your threshold

# 3. Enhanced calculation function
def calculate_rr_with_components(corp_id, eth, dis):
    corp_enroll = sch_age[sch_age['idoe_corporation_id'] == corp_id]
    corp_dis = se_dis_sch_age[se_dis_sch_age['idoe_corporation_id'] == corp_id]

    # Calculate components
    A = corp_dis[(corp_dis['ethnicity'] == eth)].shape[0]
    B = corp_enroll[corp_enroll['ethnicity'] == eth].shape[0]
    C = corp_dis[(corp_dis['ethnicity'] != eth)].shape[0]
    D = corp_enroll[corp_enroll['ethnicity'] != eth].shape[0]

    # Check criteria
    valid_A = A >= MIN_CELL_SIZE
    valid_B = B >= MIN_CELL_SIZE
    valid_C = C >= MIN_CELL_SIZE
    valid_D = D >= MIN_CELL_SIZE
    meets_criteria = all([valid_A, valid_B, valid_C, valid_D])

    # Calculate RR only if meets criteria
    rr = (A * D) / (B * C) if meets_criteria and (B * C) != 0 else float('nan')

    return {
        'A': A, 'B': B, 'C': C, 'D': D,
        'meets_criteria': meets_criteria,
        'valid_A': valid_A, 'valid_B': valid_B, 'valid_C': valid_C, 'valid_D': valid_D,
        'Risk_Ratio': rr
    }

# # 4. Process all combinations (keeping ALL results)
full_results = []
for corp_id in se_dis_sch_age['idoe_corporation_id'].unique():
    for eth, dis in product(ETHNICITY_CODES.keys(), DISABILITY_TYPES):
        res = calculate_rr_with_components(corp_id, eth, dis)
        full_results.append({
            'Corporation_ID': corp_id,
            'Ethnicity': ETHNICITY_CODES[eth],
            'Disability': dis,
            **res
        })

# # 5. Create DataFrames
all_results_df = pd.DataFrame(full_results)  # Contains ALL calculations
valid_results_df = all_results_df[all_results_df['meets_criteria']].copy()  # Only valid

# 6. Analysis reports
print("Data Quality Report:")
print(f"Total calculations: {len(all_results_df)}")
print(f"Valid calculations (meets all criteria): {len(valid_results_df)} ({len(valid_results_df)/len(all_results_df):.1%})")
print("\nBreakdown of failing criteria:")
criteria_report = all_results_df[['valid_A', 'valid_B', 'valid_C', 'valid_D']].apply(lambda x: x.value_counts())
display(criteria_report)

# 7. round risk ratio 4 after decimal, A, B, C, D should be whole number
valid_results_df = valid_results_df.round({
    'A': 0, 'B': 0, 'C': 0, 'D': 0, 'Risk_Ratio': 4
})

# Save outputs
all_results_df.to_csv('Indicator_9_RR_calculations.csv', index=False)  # All attempts
#valid_results_df.to_csv('VALID_corporation_risk_ratios.csv', index=False)  # Only valid

# 8. Optional: Create troubleshooting file
problem_cases = all_results_df[~all_results_df['meets_criteria']]
problem_cases.to_csv('corporations_needing_review.csv', index=False)


Data Quality Report:
Total calculations: 17682
Valid calculations (meets all criteria): 4398 (24.9%)

Breakdown of failing criteria:


,valid_A,valid_B,valid_C,valid_D
False,12684,8742,1866,120
True,4998,8940,15816,17562
